In [61]:
import pandas as pd
import plotly.express as px

# Load the data
df = pd.read_csv('retail_loan_hmda_bank_total_2021.csv')

# Clean the data

df = df.dropna()  # remove rows with missing values
#df['Amt_Orig'] = df['Amt_Orig'].astype(float)  # convert Amt_Orig to float
#df['Amt_Purch'] = df['Amt_Purch'].astype(float)  # convert Amt_Purch to float

# Function to retrieve and display information about a specific bank
def get_bank_info(id_rssd):
    pd.set_option('display.max_rows', None)  # display all rows
    bank_data = df[df['id_rssd'] == id_rssd]
    bank_data = bank_data.loc[:, (bank_data != 0).any(axis=0)]
    return bank_data

# Use the function
bank_info = get_bank_info(964700)
print(bank_info)

        ActivityYear          hmda_lender_id  id_rssd  State_Code  \
123488          2021  OLO8RKM2RJ2ICV6SJH34_2   964700          42   
124467          2021  OLO8RKM2RJ2ICV6SJH34_2   964700           6   
125435          2021  OLO8RKM2RJ2ICV6SJH34_2   964700          24   
126415          2021  OLO8RKM2RJ2ICV6SJH34_2   964700          24   
127846          2021  OLO8RKM2RJ2ICV6SJH34_2   964700          48   
127853          2021  OLO8RKM2RJ2ICV6SJH34_2   964700          25   
128329          2021  OLO8RKM2RJ2ICV6SJH34_2   964700           9   
128348          2021  OLO8RKM2RJ2ICV6SJH34_2   964700          25   
128811          2021  OLO8RKM2RJ2ICV6SJH34_2   964700          25   
128820          2021  OLO8RKM2RJ2ICV6SJH34_2   964700           8   
129293          2021  OLO8RKM2RJ2ICV6SJH34_2   964700          50   
129299          2021  OLO8RKM2RJ2ICV6SJH34_2   964700          25   
129775          2021  OLO8RKM2RJ2ICV6SJH34_2   964700          25   
129782          2021  OLO8RKM2RJ2I

In [94]:
# Create a new column that combines the 'State_Code' and 'County_Code' columns
df['Location'] = df['State_Code'].astype(str) + '/' + df['County_Code'].astype(str)

# Group by the new column and sum the 'Loan_Orig' column
location_loan_orig = df.groupby('Location')['Loan_Orig'].sum().reset_index()

# Sort the data by 'Loan_Orig' in descending order
location_loan_orig = location_loan_orig.sort_values('Loan_Orig', ascending=False)

# Create the bar graph
fig = px.bar(location_loan_orig, x='Location', y='Loan_Orig', title='Purchase Loans Originated by Location')

# Update the labels of the axes
fig.update_xaxes(title_text='Census Tract')
fig.update_yaxes(title_text='Loans Originated')

total_loans_originated = location_loan_orig['Loan_Orig'].sum()
print(f"Total Loans Originated: {total_loans_originated}")

fig.show()

Total Loans Originated: 933


In [69]:
#Single Fam / Multi Fam Count Purchases
df = df[df['id_rssd'] == 964700]

# Create a new DataFrame with 'Loan Type' and 'Amounts' columns
loan_types = ['1-4 Family', 'Multifamily']
amount_columns = ['Amt_Orig_SFam', 'Amt_Orig_MFam']
amounts = [df[col].sum() for col in amount_columns]
data = { 'Loan Type': loan_types, 'Amounts': amounts }
loan_data = pd.DataFrame(data)

# Add a third column for percentages
total = loan_data['Amounts'].sum()
loan_data['Percentage'] = loan_data['Amounts'] / total * 100

# Add a third row for totals
totals = {'Loan Type': 'Total', 'Amounts': total, 'Percentage': 100.0}
loan_data = loan_data._append(totals, ignore_index=True)

print(loan_data)

         Loan Type  Total Loan Amts  Percentage
0       1-4 Family         914114.0   82.518244
1      Multifamily         193658.0   17.481756
2  Total Loan Amts              NaN  100.000000


In [80]:
# Census Tract Purchases
df = df[df['id_rssd'] == 964700]

# Define the categories and corresponding columns
income_categories = ['Low', 'Moderate', 'Middle', 'Upper', 'Unknown']
loan_orig_columns = ['Loan_Orig_TILow','Loan_Orig_TIMod', 'Loan_Orig_TIMid','Loan_Orig_TIUpp','Loan_Orig_TIUnk']
loan_amt_columns = ['Amt_Orig_TILow', 'Amt_Orig_TIMod', 'Amt_Orig_TIMid', 'Amt_Orig_TIUpp', 'Amt_Orig_TIUnk']

# Calculate the sums for each category
loan_orig_sums = [df[col].sum() for col in loan_orig_columns]
loan_amt_sums = [df[col].sum() for col in loan_amt_columns]

# Calculate the total sums
total_loan_orig = sum(loan_orig_sums)
total_loan_amt = sum(loan_amt_sums)

# Calculate the percentages
loan_orig_percents = [sum_ / total_loan_orig * 100 for sum_ in loan_orig_sums]
loan_amt_percents = [sum_ / total_loan_amt * 100 for sum_ in loan_amt_sums]

# Create the DataFrame
data = {
    'Income Category': income_categories + ['Total'],
    'Loans Originated': loan_orig_sums + [total_loan_orig],
    'Percentage of Loans Originated': loan_orig_percents + [100.0],
    'Total Loan Amounts': loan_amt_sums + [total_loan_amt],
    'Percentage of Total Loan Amounts': loan_amt_percents + [100.0]
}
census_income_data = pd.DataFrame(data)

print(census_income_data)

  Income Category  Loans Originated  Percentage of Loans Originated  \
0             Low                92                        9.860665   
1        Moderate               145                       15.541265   
2          Middle               193                       20.685959   
3           Upper               499                       53.483387   
4         Unknown                 4                        0.428725   
5           Total               933                      100.000000   

   Total Loan Amounts  Percentage of Total Loan Amounts  
0            105588.0                          9.531564  
1            129758.0                         11.713421  
2            176741.0                         15.954637  
3            692428.0                         62.506364  
4              3257.0                          0.294014  
5           1107772.0                        100.000000  


In [97]:
#Borrower Income Level Purchases
df = df[df['id_rssd'] == 964700]

# Define the categories and corresponding columns
income_categories = ['Low', 'Moderate', 'Middle', 'Upper', 'Unknown']
bor_loan_orig_columns = ['Loan_Orig_BILow','Loan_Orig_BIMod', 'Loan_Orig_BIMid','Loan_Orig_BIUpp','Loan_Orig_BIUnk']
bor_loan_amt_columns = ['Amt_Orig_BILow', 'Amt_Orig_BIMod', 'Amt_Orig_BIMid', 'Amt_Orig_BIUpp', 'Amt_Orig_BIUnk']

# Calculate the sums for each category
bor_loan_orig_sums = [df[col].sum() for col in bor_loan_orig_columns]
bor_loan_amt_sums = [df[col].sum() for col in bor_loan_amt_columns]

# Calculate the total sums
bor_total_loan_orig = sum(bor_loan_orig_sums)
bor_total_loan_amt = sum(bor_loan_amt_sums)

# Calculate the percentages
bor_loan_orig_percents = [sum_ / bor_total_loan_orig * 100 for sum_ in bor_loan_orig_sums]
bor_loan_amt_percents = [sum_ / bor_total_loan_amt * 100 for sum_ in bor_loan_amt_sums]

# Create the DataFrame
bor_data = {
    'Income Category': income_categories + ['Total'],
    'Loans Originated': bor_loan_orig_sums + [bor_total_loan_orig],
    'Percentage of Loans Originated': bor_loan_orig_percents + [100.0],
    'Total Loan Amounts': bor_loan_amt_sums + [bor_total_loan_amt],
    'Percentage of Total Loan Amounts': bor_loan_amt_percents + [100.0]
}
bor_income_data = pd.DataFrame(bor_data)

print(bor_income_data)

  Income Category  Loans Originated  Percentage of Loans Originated  \
0             Low                65                        6.966774   
1        Moderate               150                       16.077170   
2          Middle               109                       11.682744   
3           Upper               513                       54.983923   
4         Unknown                96                       10.289389   
5           Total               933                      100.000000   

   Total Loan Amounts  Percentage of Total Loan Amounts  
0               15496                          1.398844  
1               41201                          3.719267  
2               39186                          3.537371  
3              711116                         64.193354  
4              300773                         27.151165  
5             1107772                        100.000000  


In [96]:
#Purchase Loans Income Categories

# Filter the data
df = df[df['id_rssd'] == 964700]

# Define the categories and corresponding columns
income_categories = ['Low', 'Moderate', 'Middle', 'Upper', 'Unknown']
bor_loan_orig_columns = ['Loan_Orig_BILow','Loan_Orig_BIMod', 'Loan_Orig_BIMid','Loan_Orig_BIUpp','Loan_Orig_BIUnk']
bor_loan_amt_columns = ['Amt_Orig_BILow', 'Amt_Orig_BIMod', 'Amt_Orig_BIMid', 'Amt_Orig_BIUpp', 'Amt_Orig_BIUnk']
loan_orig_columns = ['Loan_Orig_TILow','Loan_Orig_TIMod', 'Loan_Orig_TIMid','Loan_Orig_TIUpp','Loan_Orig_TIUnk']
loan_amt_columns = ['Amt_Orig_TILow', 'Amt_Orig_TIMod', 'Amt_Orig_TIMid', 'Amt_Orig_TIUpp', 'Amt_Orig_TIUnk']

# Calculate the sums for each category
bor_loan_orig_sums = [df[col].sum() for col in bor_loan_orig_columns]
bor_loan_amt_sums = [df[col].sum() for col in bor_loan_amt_columns]
loan_orig_sums = [df[col].sum() for col in loan_orig_columns]
loan_amt_sums = [df[col].sum() for col in loan_amt_columns]

# Calculate the total sums
bor_total_loan_orig = sum(bor_loan_orig_sums)
bor_total_loan_amt = sum(bor_loan_amt_sums)
total_loan_orig = sum(loan_orig_sums)
total_loan_amt = sum(loan_amt_sums)

# Calculate the percentages
bor_loan_orig_percents = [sum_ / bor_total_loan_orig * 100 for sum_ in bor_loan_orig_sums]
bor_loan_amt_percents = [sum_ / bor_total_loan_amt * 100 for sum_ in bor_loan_amt_sums]
loan_orig_percents = [sum_ / total_loan_orig * 100 for sum_ in loan_orig_sums]
loan_amt_percents = [sum_ / total_loan_amt * 100 for sum_ in loan_amt_sums]

# Create the DataFrames
bor_data = {
    'Income Category': income_categories + ['Total'],
    'Borrower Loans Originated': bor_loan_orig_sums + [bor_total_loan_orig],
    'Borrower Percentage of Loans Originated': bor_loan_orig_percents + [100.0],
    'Borrower Total Loan Amounts': bor_loan_amt_sums + [bor_total_loan_amt],
    'Borrower Percentage of Total Loan Amounts': bor_loan_amt_percents + [100.0]
}
bor_income_data = pd.DataFrame(bor_data)

census_data = {
    'Income Category': income_categories + ['Total'],
    'Census Loans Originated': loan_orig_sums + [total_loan_orig],
    'Census Percentage of Loans Originated': loan_orig_percents + [100.0],
    'Census Total Loan Amounts': loan_amt_sums + [total_loan_amt],
    'Census Percentage of Total Loan Amounts': loan_amt_percents + [100.0]
}
census_income_data = pd.DataFrame(census_data)

# Combine the DataFrames
combined_data = pd.concat([bor_income_data.set_index('Income Category'), census_income_data.set_index('Income Category')], axis=1).reset_index()

print(combined_data)

  Income Category  Borrower Loans Originated  \
0             Low                         65   
1        Moderate                        150   
2          Middle                        109   
3           Upper                        513   
4         Unknown                         96   
5           Total                        933   

   Borrower Percentage of Loans Originated  Borrower Total Loan Amounts  \
0                                 6.966774                        15496   
1                                16.077170                        41201   
2                                11.682744                        39186   
3                                54.983923                       711116   
4                                10.289389                       300773   
5                               100.000000                      1107772   

   Borrower Percentage of Total Loan Amounts  Census Loans Originated  \
0                                   1.398844                    

In [93]:
loan_types = ['Home Improvement', 'Home Purchase', 'Multi-Family Purchase', 'Refinance', 'Total']
loan_orig_columns = ['Loan_Orig_HI', 'Loan_Orig_SFam', 'Loan_Orig_MFam', 'Loan_Orig_HR']
loan_amt_columns = ['Amt_Orig_HI', 'Amt_Orig_SFam', 'Amt_Orig_MFam', 'Amt_Orig_HR']

# Calculate the sums for each loan type
loan_orig_sums = [df[col].sum() for col in loan_orig_columns]
loan_amt_sums = [df[col].sum() for col in loan_amt_columns]

# Calculate the total sums
total_loan_orig = sum(loan_orig_sums)
total_loan_amt = sum(loan_amt_sums)

# Add the total sums to the lists
loan_orig_sums.append(total_loan_orig)
loan_amt_sums.append(total_loan_amt)

# Calculate the percentages
loan_orig_percents = [sum_ / total_loan_orig * 100 for sum_ in loan_orig_sums]
loan_amt_percents = [sum_ / total_loan_amt * 100 for sum_ in loan_amt_sums]

# Create the DataFrame
data = {
    'Loan Type': loan_types,
    'Loans Originated': loan_orig_sums,
    'Percentage of Loans Originated': loan_orig_percents,
    'Total Loan Amounts': loan_amt_sums,
    'Percentage of Total Loan Amounts': loan_amt_percents
}
loan_data = pd.DataFrame(data)

print(loan_data)

               Loan Type  Loans Originated  Percentage of Loans Originated  \
0       Home Improvement                 9                        0.647017   
1          Home Purchase               897                       64.485981   
2  Multi-Family Purchase                36                        2.588066   
3              Refinance               449                       32.278936   
4                  Total              1391                      100.000000   

   Total Loan Amounts  Percentage of Total Loan Amounts  
0               15334                          0.893138  
1              914114                         53.243115  
2              193658                         11.279726  
3              593762                         34.584022  
4             1716868                        100.000000  


In [ ]:
#Assessment Area Lending
